
 # Building the recommendation engine

Obstajajo trie glavni principi grajenja modelov za priporočanje:
 
     1. Modeli, ki temeljijo na splošni priljubljenosti
     2. Modeli, ki temeljijo na vsebini
     3. Modeli, ki temeljijo na ocenah podobnih uporabnikov

 
 ### Modeli, ki temeljijo na splošni priljubljenosti:
 
 To je najpreprostejša oblika modelov. Trendi se zaznavajo s pomočjo štetj ogledov/klikov, in so na podlagi tega razvrščeni po priljubljenosti (YouTube, Netflix,...).
 
 ### Modeli, ki temeljijo na vsebini:
 
 Ti modeli kot vhodni podatek vzamejo film, ki je všeč uporabniku. Potm preučijo njegovo vsebino filma (zgodba, žanr, igralska zasedba, režiser,...) in priporoči filme, ki so po teh parametrih podobni izbranemu filmu. Uporabniku priporoča filme in jih lahko razvrsti po relavantnosti glede na želen kriterij.
 
 ### Modeli, ki temeljijo na ocenah podobnih uporabnikov:
 
 Ti modeli najprej poizkušajo poiskati podobne uporabnike glede na njihove preference (npr. oba uporabnika gledata filme istega žanra, istega režiserja,...). Potem med sorodnima uporabnikoma A in B priporoča filme, ki jih eden od njiju še ni videl, drugemu pa je bil všeč. Priporočila so torej filtrirana glede na preference sorodnih uporabnikov. Takšni modeli se uporabljajo na Amazonu ("Customers who viewed this item also viewed","Customers who bought this item also bought" ).

 https://miro.medium.com/max/875/0*wJ1hfDhXNYJbk8Aj

 Mi bomo konstruirali model, ki temelji na vsebini.

 ## Iskanje podobnosti
 
 *We know that our recommendation engine will be content based. So, we need to find similar movies to a given movie and then recommend those similar movies to the user. The logic is pretty straightforward. Right?
 
 But, wait…. How can we find out which movies are similar to the given movie in the first place? How can we find out how much similar(or dissimilar) two movies are?
 
 Let us start with something simple and easy to understand.*
 
 Imamo dve besedili:
 
 Besedilo A: Jabolko Banana Banana
 
 Besedilo B: Banana Jabolko Jabolko
 
 Kako najdemo podobnosti med zgornjima primeroma?
 
 Če analiziramo besedili….
 
     Besedilo A: Vsebuje dve ponovitvi besede "Banana" in eno ponovitev besede "Jabolko".
     Besedilo B: Vsebuje dve ponovitvi besede "Jabolko" in eno ponovitev besede "Banana".
 
 Predstavimo besedili v ravnini, kjer bo na x-osi št. ponovitv besede "Banana" in na y-osi št. ponovitev besede "Jabolko". Vsako od besedil je tako predstavljeno kot vektor.

 https://miro.medium.com/max/875/0*Von8RtqkHK-WD1w4

 Rumen vektor predstavlja besedilo A, rdeč pa besedilo B.

 Ali lahko s pomočjo vektorjev in matematike določimo, ali sta si besedili podobni?

 
 *Now we have graphically represented these two texts. So, now can we find out the similarity between these two texts?
 
 The answer is "Yes, we can". But, exactly how?
 
 These two texts are represented as vectors. Right? So, we can say that two vectors are similar if the distance between them is small. By distance, we mean the angular distance between two vectors, which is represented by θ (theta). By thinking further from the machine learning perspective, we can understand that the value of cos θ makes more sense to us rather than the value of θ (theta) because, the cosine(or "cos") function will map the value of θ in the first quadrant between 0 to 1 (Remember? cos 90° = 0 and cos 0° = 1 ).
 
 And from high school maths, we can remember that there is actually a formula for finding out cos θ between two vectors.*  See the picture below-

 Kako pa bi to naredili v Pythonu?
 
 Najprej moramo shraniti naši besedili v seznam:



In [ ]:
besedili = ['Jabolko Banana Banana','Banana Jabolko Jabolko']

Kako bomo predstavili besedili v seznamu kot vektorja na sliki? . Razred [`CountVectorizer()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) iz knjižnice `sklearn.feature_extraction.text` naredi prav to. Najprej pa moramo seveda uvoziti knižnico, preden lahko naredimo `CountVectorizer()` objekt.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
count_matrix = cv.fit_transform(besedili)

print(count_matrix)

`count_matrix` nam vrne razpršeno matriko. Da pa bo oblika nam bolj razumljiva, jo prevedemo v array z metodo `toarrray()`. Še pred tem pa bomo vrnili seznam lastnosti (feature list), ki smo ga podali objektu `CountVectorizer()`.

In [ ]:
print(cv.get_feature_names())
print(count_matrix.toarray())

 *This indicates that the word 'banana' occurs 2 times in A and 1 time in B. Similarly, the word 'jabolko' occurs 1 time in A and 2 times in B. Makes sense. Right?*
Zgornji array nam je podal besedili v izomorfni obliki, kot smo jih prikazali na sliki.
Zdaj pa izračunajmo še kosinus kota med vektorjema, da dobimo numerično opredelitev, kako blizu sta si vektorja oziroma naši besedili. To bomo izračunali s pomočjo funkcije `cosine_similarity()` iz knjižnice `sklearn.metrics.pairwise`.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(count_matrix)
print(similarity_scores)

 Kaj nam pove vrnjena matrika podobnosti?
 
 Interpretiramo jo na sledeč način
 
         1. Vsaka vrstica matrike podobnosti predstavlja enega od naših besedil. Torej vrstica 0 = Besedilo A in vrstica 1 = Besedilo B.
     2. Enako velja za stolpca.
     
 Bolj pregleden zapis bi bil morda takle:

In [ ]:
Besedilo A:     Besedilo B:
Besedilo A: [[1.        0.8]
Besedilo B: [0.8        1.]]

Matriko lahko razumemo kot tabelo, ki nam poda podobnost med dvema besediloma. Število 1. na mestu [0, 0] je podobnost Besedila A s samim sabo, kar pomeni, da je ujemanje 100%. Prav tako velja za ujemanje Besedila B s samim sabo (število na mestu [1, 1]). Ujemanje med Besedilom A in B je enako, kot če vrstni red zamenjamo, in je 80% (števili 0.8 na mestih [0, 1] in [1, 0]). Vidimo, da je zato matrika podobnosti zmeraj simetrična.
 
 Zdaj vemo, kako poiskati podobnost med dvema vsebinama. Sledi implementacija za naš začetni problem - model za priporočanje filmov.

## The recommendation engine

 After downloading [the dataset](https://github.com/codeheroku/Introduction-to-Machine-Learning/blob/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv), we need to import all the required libraries and then read the csv file using `read_csv()` method.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
df = pd.read_csv("C:\\Users\\asmerdu\\Documents\\movies_db.csv")

Pogejmo si podatke, ki jih imamo:

In [ ]:
#df.shape
#df.size
#df.values
 df.head(10)

Vidimo, da je za posameze film mnogo podatkov, ki za nas niso tako zanimivi. Izmet vseh lahko izberemo tiste, ki se nam zdijo najbolj pomembni. To ne naša množica lastnosti (*feature set*).


In [ ]:
features = ['keywords','cast','genres','director']

Naslednja naloga je, da napišemo funkcijo, ki bo združila vrednosti vsake vrstice v niz.

In [ ]:
def combine_features(row):
 return row['keywords']+" "+row['cast']+" "+row["genres"]+" "+row["director"]

To funkcijo bomo poklicali na vsaki vrtici iz naše množice podatkov. Še prej bomo s pomočjo for zanke nadomestili vse neveljavne vnose [`NaN`](https://en.wikipedia.org/wiki/NaN) s praznim nizom.


In [ ]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string
    
df["combined_features"] = df.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column

Zdaj, ko imamo pripravljene prečiščene podatke, lahko nadaljujemo `CountVectorizer()`, da bomo dobili matriko vektorjev.


In [ ]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object


Izračunamo matriko podobnosti.

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

Definirali bomo funkciji, ki povežeta naslov filma in njegov indeks (in obratno).

In [ ]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

Zdaj lahko vnesemo ime filma, ki nam je trenutno všeč. Našli bomo njegov indeks, da bomo lahko dostopali do ustrezne vrstice v matriki podobnosti. Tako bomo lhko prebrali podobnosti izbranega filma z drugimi filmi. Sprehodili se bomo čez vse ocene podobnosti in jih shranili v seznam urejenih parov, skupaj z zaporedno številko. Zgledalo bo približno tako: vrstico ocen iz matrike podobnosti `[1 0.5 0.2 0.9]` bo preoblikovalo v  seznam parov `[(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)]`. Prva številka v paru je indeks filma, druga pa ocena podobnosti.


In [ ]:
movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it

 Now comes the most vital point. We will sort the list `similar_movies` according to similarity scores in descending order. Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.

In [ ]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

S for zanko se bomo sprehodili čez urejen seznam filmov po podobnosti in izpisali 5 najbolj podobnih filmov.

In [ ]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Če zapišemo kodo na enem mestu, bi zgledala takole:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("C:\\Users\\asmerdu\\Documents\\movies_db.csv")

features = ['keywords','cast','genres','director']

def combine_features(row):
    return row['keywords'] +" "+row['cast']+" "+row["genres"]+" "+row["director"]

for feature in features:
    df[feature] = df[feature].fillna('')

df["combined_features"] = df.apply(combine_features,axis=1)

cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])

cosine_sim = cosine_similarity(count_matrix)

def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies =  list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>=5:
        break